In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from pycocotools.coco import COCO

In [5]:
TACO_DIR = Path("/Users/hariomnarang/Desktop/personal/TACO/data")
ANN_FILE = TACO_DIR / "annotations.json"
OUT = Path("../../datasets/taco_with_seg_masks")

In [6]:
from mtrain.smallnet.unet.extract import _extract_single_crop

In [6]:
! ls ../../datasets/taco_with_seg_masks/images

0.jpeg    1167.jpeg 1336.jpeg 156.jpeg  326.jpeg  497.jpeg  666.jpeg  835.jpeg
1.jpeg    1168.jpeg 1337.jpeg 157.jpeg  327.jpeg  498.jpeg  667.jpeg  836.jpeg
10.jpeg   1169.jpeg 1338.jpeg 158.jpeg  328.jpeg  499.jpeg  668.jpeg  837.jpeg
100.jpeg  117.jpeg  1339.jpeg 159.jpeg  329.jpeg  5.jpeg    669.jpeg  838.jpeg
1000.jpeg 1170.jpeg 134.jpeg  16.jpeg   33.jpeg   50.jpeg   67.jpeg   839.jpeg
1001.jpeg 1171.jpeg 1340.jpeg 160.jpeg  330.jpeg  500.jpeg  670.jpeg  84.jpeg
1002.jpeg 1172.jpeg 1341.jpeg 161.jpeg  331.jpeg  501.jpeg  671.jpeg  840.jpeg
1003.jpeg 1173.jpeg 1342.jpeg 162.jpeg  332.jpeg  502.jpeg  672.jpeg  841.jpeg
1004.jpeg 1174.jpeg 1343.jpeg 163.jpeg  333.jpeg  503.jpeg  673.jpeg  842.jpeg
1005.jpeg 1175.jpeg 1344.jpeg 164.jpeg  334.jpeg  504.jpeg  674.jpeg  843.jpeg
1006.jpeg 1176.jpeg 1345.jpeg 165.jpeg  335.jpeg  505.jpeg  675.jpeg  844.jpeg
1007.jpeg 1177.jpeg 1346.jpeg 166.jpeg  336.jpeg  506.jpeg  676.jpeg  845.jpeg
1008.jpeg 1178.jpeg 1347.jpeg 167.jpeg  337.jpeg  507

In [ ]:
from mtrain.smallnet.unet.extract import _corres_mask_name, Progress, Image, _save_mask, chunk_list
from multiprocessing.dummy import Pool as ThreadPool

class MpCropHelper:
    def __init__(self, input_masks_dir, tile_size, max_padding, coco, output_images_dir, output_masks_dir):
        self.input_masks_dir = input_masks_dir
        self.tile_size = tile_size
        self.max_padding = max_padding
        self.coco = coco
        self.output_images_dir = output_images_dir
        self.output_masks_dir = output_masks_dir

    def __call__(self, i_and_chunk):
        chunk_id, paths = i_and_chunk
        progress =Progress(len(paths), f"MPCrop {chunk_id}:")
        res = []
        for i, img_path in enumerate(paths):
            mask_path = self.input_masks_dir / _corres_mask_name(img_path)

            # Generate crop
            progress(i)
            try:
                crop_img, crop_mask, _ = _extract_single_crop(
                    self.coco, img_path, mask_path, self.tile_size, max_padding=self.max_padding
                )
                # res.append((img_path, crop_img, crop_mask))
                img_fname = img_path.name
                Image.fromarray(crop_img).save(self.output_images_dir / img_fname)
                _save_mask(crop_mask, self.output_masks_dir / _corres_mask_name(img_fname))
            except Exception as ex:
                print(f"failure in generating crop for {img_path}, reason={ex}. ignoring")
                continue

            # Save with random filename
        with ThreadPool(24) as p:
            def _save_images(reses):
                for (img_path, crop_img, crop_mask) in reses:
                    img_fname = img_path.name
                    Image.fromarray(crop_img).save(self.output_images_dir / img_fname)
                    _save_mask(crop_mask, self.output_masks_dir / _corres_mask_name(img_fname))
            p.apply_async(_save_images, chunk_list(res))


In [9]:
from mtrain.smallnet.unet.extract import MpCropHelper
from mtrain.smallnet.unet.extract import _corres_mask_name, Progress, Image, _save_mask, chunk_list

test_out_images = Path("./test-outs/images")
test_out_masks = Path("./test-outs/masks")
test_out_images.mkdir(parents=True, exist_ok=True)
test_out_masks.mkdir(parents=True, exist_ok=True)


coco = COCO(ANN_FILE)
helper = MpCropHelper(OUT / "masks", 50, 1000, coco, test_out_images, test_out_masks)

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!


In [10]:
chunks = chunk_list(list((OUT / "images").glob("*")), 8)

In [11]:
len(chunks[0])

188

In [12]:
chunk = list((OUT / "images").glob("*"))[:100]

In [14]:
%%time
helper((0, chunks[0]))

Progress track: MPCrop 0: total_samples=188
MPCrop 0:: 0%
MPCrop 0:: 10%
failure in generating crop for ../../datasets/taco_with_seg_masks/images/298.jpeg, reason=OpenCV(4.13.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
. ignoring
MPCrop 0:: 20%
MPCrop 0:: 30%
MPCrop 0:: 40%
MPCrop 0:: 50%
MPCrop 0:: 60%
MPCrop 0:: 70%
MPCrop 0:: 80%
MPCrop 0:: 90%
MPCrop 0:: 100%
CPU times: user 20.6 s, sys: 3.07 s, total: 23.7 s
Wall time: 22.7 s


In [39]:
%%time
helper((0, chunks[0]))

CPU times: user 16.8 s, sys: 4.77 s, total: 21.6 s
Wall time: 22.4 s


In [25]:
len(chunks)

8

In [26]:
len(chunks[0])

188

In [ ]:
%%time
from multiprocessing.dummy import Pool

with Pool(16) as p:
    p.map(helper, enumerate(chunks[:8]))

Progress track: MPCrop 0: total_samples=188Progress track: MPCrop 1: total_samples=188

Progress track: MPCrop 2: total_samples=187
Progress track: MPCrop 3: total_samples=187
Progress track: MPCrop 4: total_samples=187
Progress track: MPCrop 5: total_samples=187
Progress track: MPCrop 6: total_samples=187
Progress track: MPCrop 7: total_samples=187
MPCrop 3:: 0%
MPCrop 5:: 0%MPCrop 1:: 0%

MPCrop 7:: 0%
MPCrop 4:: 0%
MPCrop 6:: 0%
MPCrop 0:: 0%
MPCrop 2:: 0%
MPCrop 7:: 10%
MPCrop 0:: 10%
MPCrop 5:: 10%MPCrop 3:: 10%

MPCrop 6:: 10%
MPCrop 1:: 10%
MPCrop 4:: 10%
MPCrop 2:: 10%
failure in generating crop for ../../datasets/taco_with_seg_masks/images/572.jpeg, reason=OpenCV(4.13.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
. ignoring
MPCrop 6:: 20%
MPCrop 7:: 20%
MPCrop 3:: 20%
MPCrop 0:: 20%
MPCrop 4:: 20%MPCrop 1:: 20%

MPCrop 5:: 20%
MPCrop 2:: 20%

In [ ]:
import itertools

it = itertools.islice(itertools.cycle((OUT / "images").glob("*")), 5000)
images = list(it)

In [7]:
coco = COCO(ANN_FILE)
img_path = OUT / "images" / "0.jpeg"
mask_path = OUT / "masks" / "0.png"

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!


In [8]:
%%time
_extract_single_crop(coco, img_path, mask_path, 50, max_padding=1000)

CPU times: user 70 ms, sys: 21.4 ms, total: 91.4 ms
Wall time: 122 ms


(array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        ...,
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]], shape=(50, 50, 3), dtype=uint8),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, .